## Análisis de datos - Clase 3

### Taller práctico - análisis completo de un dataset

En esta clase vamos a analizar en forma completa un dataset con datos de Fórmula 1 de la temporada 2024 (los datos fueron extraídos con la biblioteca Fastf1). El dataset contiene información sobre las carreras, los pilotos, los tiempos y los resultados de cada evento.


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

1. Importamos los datasets

In [20]:
# Dataset de carreras del 2024
carreras_24 = pd.read_csv('../datasets/f1_2024_R.csv')

# Dataset de qualifications del 2024
qualis_24 = pd.read_csv('../datasets/f1_2024_Q.csv')

# Dataset de sprints del 2024
sprints_24 = pd.read_csv('../datasets/f1_2024_S.csv')

In [21]:
carreras_24.shape, qualis_24.shape, sprints_24.shape

((26606, 34), (7590, 34), (2418, 34))

In [25]:
carreras_24.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26606 entries, 0 to 26605
Data columns (total 34 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Time                26606 non-null  object 
 1   Driver              26606 non-null  object 
 2   DriverNumber        26606 non-null  int64  
 3   LapTime             26381 non-null  float64
 4   LapNumber           26606 non-null  float64
 5   Stint               26606 non-null  float64
 6   PitOutTime          840 non-null    object 
 7   PitInTime           849 non-null    object 
 8   Sector1Time         26066 non-null  float64
 9   Sector2Time         26574 non-null  float64
 10  Sector3Time         26553 non-null  float64
 11  Sector1SessionTime  26008 non-null  object 
 12  Sector2SessionTime  26574 non-null  object 
 13  Sector3SessionTime  26553 non-null  object 
 14  SpeedI1             22460 non-null  float64
 15  SpeedI2             26552 non-null  float64
 16  Spee